In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LeakyReLU, Flatten, MaxPooling1D, Conv1D, MaxPool1D, GlobalMaxPool1D, Dense, AveragePooling1D, GlobalAveragePooling1D

In [2]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
%matplotlib tk

In [3]:
import os, os.path, sys

In [4]:
path = r"C:\Users\mata2\Desktop\master\podaci\0k\X"

In [5]:
dir = os.listdir(path)

Ucitavanje fajlova

In [6]:
file_names = []
for file in dir:
    file_names.append(file)

In [7]:
percent_train = 0.8

In [8]:
percent_val = 0.2

Konkatenacija podataka koji se posle koriste za adekvatnu raspodelu na skup za treniranje i validaciju

In [9]:
train_files_names = file_names[:int(percent_train*len(file_names))]
#test_file_names = file_names[int(percent_train*len(file_names))::]

Ucitavanje zadate trajektorije

In [10]:
x_command = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\xCmd.txt",header=None).astype('int'))

In [11]:
train_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in train_files_names], axis=0)
# test_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
#                               for file in test_file_names], axis=0)

Ucitavanje stadardnih devijacija i strednih vrednosti

In [12]:
std_dev = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k\std_dev.txt", 'r') as lines:
    for line in lines:
        std_dev.append(float(line.strip('\n')))

In [13]:
sr_vrednost = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k\sr_vrednost.txt", 'r') as lines:
    for line in lines:
        sr_vrednost.append(float(line.strip('\n')))

Raspodela podataka kako bi se izbegla fiktivna stanja

In [15]:
for j in range(len(train_files_names)):
    exec(f"razlika_trening_{j} = []")
    for i in range(len(x_command)):
        exec(f"razlika_trening_{j}.append(x_command[i] - train_[j*len(x_command)+i])")
    exec(f'razlika_trening_{j} = np.asarray(razlika_trening_{j})')        

In [14]:
def create_timeseries_split(data, time_steps):

    # Get the number of samples in the data
    num_samples = len(data) - time_steps

    # Create empty arrays to store features and target values
    x_train = np.zeros((num_samples, time_steps, 1))
    y_train = np.zeros((num_samples, 1))

    # Loop through the data and create features and target values
    for i in range(num_samples):
        # Extract a slice of data for the current feature
        x_train[i] = data[i:i+time_steps, :]

        # The target value is the next value after the feature
        y_train[i] = data[i+time_steps, 0]

    return x_train, y_train

In [55]:
time_steps = 8
input_shape = (time_steps,1)

In [56]:
for i in range(len(train_files_names)):
    exec(f'train_X_{i}, train_y_{i} = create_timeseries_split(razlika_trening_{i}, time_steps)')
    
train_X = np.asarray(train_X_0)
for i in range(1,len(train_files_names)):
    exec(f'train_X = np.append(train_X, train_X_{i}, axis=0)')

train_y = np.asarray(train_y_0)
for i in range(1,len(train_files_names)):
    exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)') 

In [57]:
val_x = train_X[-int(len(train_X)*percent_val)-1:-1]
val_y = train_y[-int(len(train_X)*percent_val)-1:-1]

In [58]:
train_X = train_X[:len(train_X)-val_x.shape[0]]
train_y = train_y[:len(train_y)-val_x.shape[0]]

# TESTIRANJE ARHITEKTURE i LR/BATCH

In [36]:
filter_size = [16,32]
kernel_size_1 = [3,4]
kernel_size_2 = [3,4]
pooling_size = [3,4]
output_dens = [16,32]
#batch_size = [256]
time_step_list = [8,10]

In [37]:
best_skor = float('-inf')
best_filter = None
best_kernel_1 = None
best_kernel_2 = None
best_pool = None
#best_batch = None
best_out = None
best_step = None
loop = 0
for step in time_step_list:
    
    #delimo podatke na osnovu velicine buffera
    input_shape = (step,1)

    for i in range(len(train_files_names)):
        exec(f'train_X_{i}, train_y_{i} = create_timeseries_split(razlika_trening_{i}, step)')

    train_X = np.asarray(train_X_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_X = np.append(train_X, train_X_{i}, axis=0)')

    train_y = np.asarray(train_y_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)')  
    
    #uzimamo percent_val sa kraja train_X skupa 
    val_x = train_X[-int(len(train_X)*percent_val)-1:-1]
    val_y = train_y[-int(len(train_X)*percent_val)-1:-1]

    train_X = train_X[:len(train_X)-val_x.shape[0]]
    train_y = train_y[:len(train_y)-val_x.shape[0]]
    for filt in filter_size:
        for kernel_1 in kernel_size_1:
            for kernel_2 in kernel_size_2:
                for pool in pooling_size:
                    for out in output_dens:
                        keras.utils.set_random_seed(7)
                        optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)
                        loop += 1

                        modelMSE = Sequential()
                        modelMSE.add(Input(shape=input_shape))
                        modelMSE.add(Conv1D(filters=filt, kernel_size=kernel_1,padding='same', activation=LeakyReLU()))
                        #modelMSE.add(Conv1D(filters=32, kernel_size=2, activation=LeakyReLU()))
                        modelMSE.add(MaxPooling1D(pool_size=pool))
                        modelMSE.add(Conv1D(filters=filt, kernel_size=kernel_2,padding='same', activation=LeakyReLU()))
                        #modelMSE.add(MaxPooling1D(pool_size=pool))
                        modelMSE.add(Flatten())
                        modelMSE.add(Dense(out, activation=LeakyReLU()))
                        modelMSE.add(Dense(units = 1, activation=LeakyReLU()))
                        modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

                        modelMSE.fit(train_X, train_y, epochs = 10, batch_size = 256, verbose = 2, shuffle = False)
                        test_predict_MSE = modelMSE.predict(val_x)



                        #zato sto smo uzeli 20% od kraja, ne pocinje indeks od samog pocetka rada masine
                        #radimo evaluaciju na mean 3 sigma testu
                        i =(len(x_command)-step) - val_x.shape[0]%(len(x_command)-step) -1
                        mean_3std = 0
                        for j in range(len(test_predict_MSE)):
                            if i%len(x_command) == 0:
                                i = step
                            if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
                                    mean_3std += 1
                            i += 1



                        if mean_3std > best_skor:
                            best_skor = mean_3std
                            best_filter = filt
                            best_kernel_1 = kernel_1
                            best_kernel_2 = kernel_2
                            best_pool = pool
                            #best_batch = batch
                            best_out = out
                            best_step = step


                            print('best_skor: ', best_skor)
                            print('best_filter: ', best_filter)
                            print('best_kernel_1: ', best_kernel_1)
                            print('best_kernel_2: ', best_kernel_2)
                            print('best_out: ', best_out )
                            #print('best_batch', best_batch)
                            print('best_step', best_step)
                            print('best_pool', best_pool)

                        print('loop: ', loop)

print('best_skor: ', best_skor)
print('best_filter: ', best_filter)
print('best_kernel_1: ', best_kernel_1)
print('best_kernel_2: ', best_kernel_2)
print('best_out: ', best_out )
#print('best_batch', best_batch)
print('best_step', best_step)
print('best_pool', best_pool)

Epoch 1/10
4575/4575 - 10s - loss: 14.4612 - 10s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 9s - loss: 9.2360 - 9s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 9s - loss: 8.2729 - 9s/epoch - 2ms/step
Epoch 4/10
4575/4575 - 9s - loss: 7.8083 - 9s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 9s - loss: 7.4469 - 9s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 9s - loss: 7.1723 - 9s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 9s - loss: 6.9666 - 9s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 9s - loss: 6.7939 - 9s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 9s - loss: 6.6499 - 9s/epoch - 2ms/step
Epoch 10/10
4575/4575 - 9s - loss: 6.5280 - 9s/epoch - 2ms/step
9149/9149 [==============================] - 10s 1ms/step
best_skor:  289158
best_filter:  16
best_kernel_1:  3
best_kernel_2:  3
best_out:  16
best_step 8
best_pool 3
loop:  1
Epoch 1/10
4575/4575 - 10s - loss: 13.8519 - 10s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 9s - loss: 8.8749 - 9s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 9s - loss: 8.0805 - 9s/epoch - 2ms/st

Epoch 4/10
4575/4575 - 9s - loss: 3.3207 - 9s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 9s - loss: 3.0328 - 9s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 9s - loss: 2.8516 - 9s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 9s - loss: 2.7324 - 9s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 9s - loss: 2.6496 - 9s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 9s - loss: 2.5867 - 9s/epoch - 2ms/step
Epoch 10/10
4575/4575 - 9s - loss: 2.5356 - 9s/epoch - 2ms/step
9149/9149 [==============================] - 10s 1ms/step
loop:  12
Epoch 1/10
4575/4575 - 10s - loss: 11.0607 - 10s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 9s - loss: 4.5894 - 9s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 9s - loss: 3.5262 - 9s/epoch - 2ms/step
Epoch 4/10
4575/4575 - 9s - loss: 3.0610 - 9s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 9s - loss: 2.8063 - 9s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 9s - loss: 2.6502 - 9s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 9s - loss: 2.5507 - 9s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 9s - loss: 2.4796 - 9s/

Epoch 10/10
4575/4575 - 10s - loss: 2.3772 - 10s/epoch - 2ms/step
9149/9149 [==============================] - 10s 1ms/step
loop:  23
Epoch 1/10
4575/4575 - 11s - loss: 8.0108 - 11s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 10s - loss: 3.6277 - 10s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 10s - loss: 2.9671 - 10s/epoch - 2ms/step
Epoch 4/10
4575/4575 - 10s - loss: 2.7464 - 10s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 10s - loss: 2.6169 - 10s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 10s - loss: 2.5310 - 10s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 10s - loss: 2.4680 - 10s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 10s - loss: 2.4219 - 10s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 10s - loss: 2.3807 - 10s/epoch - 2ms/step
Epoch 10/10
4575/4575 - 10s - loss: 2.3474 - 10s/epoch - 2ms/step
9149/9149 [==============================] - 10s 1ms/step
loop:  24
Epoch 1/10
4575/4575 - 11s - loss: 9.0823 - 11s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 10s - loss: 4.0943 - 10s/epoch - 2ms/step
Epoch 3/10
4575/4

Epoch 4/10
4575/4575 - 8s - loss: 8.2892 - 8s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 8s - loss: 7.8413 - 8s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 8s - loss: 7.4740 - 8s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 9s - loss: 7.1859 - 9s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 8s - loss: 6.9704 - 8s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 8s - loss: 6.8032 - 8s/epoch - 2ms/step
Epoch 10/10
4575/4575 - 8s - loss: 6.6710 - 8s/epoch - 2ms/step
9149/9149 [==============================] - 10s 1ms/step
loop:  35
Epoch 1/10
4575/4575 - 10s - loss: 13.6300 - 10s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 9s - loss: 9.3961 - 9s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 9s - loss: 8.2657 - 9s/epoch - 2ms/step
Epoch 4/10
4575/4575 - 9s - loss: 7.8175 - 9s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 9s - loss: 7.5437 - 9s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 9s - loss: 7.3268 - 9s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 9s - loss: 7.1453 - 9s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 9s - loss: 6.9804 - 9s/

9149/9149 [==============================] - 10s 1ms/step
loop:  46
Epoch 1/10
4575/4575 - 10s - loss: 11.9147 - 10s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 8s - loss: 5.4117 - 8s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 9s - loss: 4.4299 - 9s/epoch - 2ms/step
Epoch 4/10
4575/4575 - 8s - loss: 3.9548 - 8s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 9s - loss: 3.5272 - 9s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 8s - loss: 3.1831 - 8s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 8s - loss: 2.9620 - 8s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 8s - loss: 2.8258 - 8s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 8s - loss: 2.7336 - 8s/epoch - 2ms/step
Epoch 10/10
4575/4575 - 8s - loss: 2.6638 - 8s/epoch - 2ms/step
9149/9149 [==============================] - 10s 1ms/step
loop:  47
Epoch 1/10
4575/4575 - 10s - loss: 10.6097 - 10s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 9s - loss: 5.2126 - 9s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 9s - loss: 4.2889 - 9s/epoch - 2ms/step
Epoch 4/10
4575/4575 - 9s - loss: 3.81

Epoch 5/10
4575/4575 - 10s - loss: 2.3634 - 10s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 10s - loss: 2.2700 - 10s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 10s - loss: 2.2032 - 10s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 10s - loss: 2.1500 - 10s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 10s - loss: 2.1127 - 10s/epoch - 2ms/step
Epoch 10/10
4575/4575 - 10s - loss: 2.0788 - 10s/epoch - 2ms/step
9149/9149 [==============================] - 10s 1ms/step
loop:  58
Epoch 1/10
4575/4575 - 11s - loss: 9.9428 - 11s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 10s - loss: 4.8605 - 10s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 10s - loss: 3.7941 - 10s/epoch - 2ms/step
Epoch 4/10
4575/4575 - 10s - loss: 3.2056 - 10s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 10s - loss: 2.8506 - 10s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 10s - loss: 2.6587 - 10s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 10s - loss: 2.5466 - 10s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 10s - loss: 2.4706 - 10s/epoch - 2ms/step
Epoch 9/10
4575/4575 

In [64]:
keras.utils.set_random_seed(7)
optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

modelMSE = Sequential()
modelMSE.add(Input(shape=(time_steps, 1)))
modelMSE.add(Conv1D(filters=16, kernel_size=3,padding='same', activation=LeakyReLU()))
modelMSE.add(MaxPooling1D(pool_size=3))
modelMSE.add(Conv1D(filters=16, kernel_size=4,padding='same', activation=LeakyReLU()))
#modelMSE.add(GlobalAveragePooling1D())
modelMSE.add(Flatten())
modelMSE.add(Dense(32, activation=LeakyReLU()))
modelMSE.add(Dense(units = 1, activation=LeakyReLU()))
modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

In [65]:
historyMSE = modelMSE.fit(train_X, train_y, epochs=10, batch_size = 256, shuffle=False, verbose = 2)

Epoch 1/10
4575/4575 - 8s - loss: 14.5834 - 8s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 7s - loss: 9.7455 - 7s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 7s - loss: 8.7313 - 7s/epoch - 2ms/step
Epoch 4/10
4575/4575 - 7s - loss: 8.3074 - 7s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 7s - loss: 8.0064 - 7s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 7s - loss: 7.7729 - 7s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 7s - loss: 7.5754 - 7s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 8s - loss: 7.3987 - 8s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 8s - loss: 7.2362 - 8s/epoch - 2ms/step
Epoch 10/10
4575/4575 - 8s - loss: 7.0918 - 8s/epoch - 2ms/step


In [44]:
modelMSE.summary()

Model: "sequential_137"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_210 (Conv1D)         (None, 8, 16)             64        
                                                                 
 max_pooling1d_140 (MaxPool  (None, 2, 16)             0         
 ing1D)                                                          
                                                                 
 conv1d_211 (Conv1D)         (None, 2, 16)             1040      
                                                                 
 flatten_136 (Flatten)       (None, 32)                0         
                                                                 
 dense_272 (Dense)           (None, 32)                1056      
                                                                 
 dense_273 (Dense)           (None, 1)                 33        
                                                    

In [67]:
test_predict_MSE = modelMSE.predict(val_x)

9149/9149 [==============================] - 9s 1ms/step


In [66]:
i =(len(x_command)-time_steps) - val_x.shape[0]%(len(x_command)-time_steps) -1
mean_3std = 0
for j in range(len(test_predict_MSE)):
    if i%len(x_command) == 0:
        i = time_steps
    if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
            mean_3std += 1
    i += 1
mean_3std/len(test_predict_MSE)

0.9814729420926968

##  jednostavnija arhitektura

In [49]:
keras.utils.set_random_seed(7)
optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

modelMSE = Sequential()
modelMSE.add(Input(shape=(time_steps, 1)))
modelMSE.add(Conv1D(filters=8, kernel_size=3,padding='same', activation=LeakyReLU()))
modelMSE.add(MaxPooling1D(pool_size=3))
modelMSE.add(Flatten())
modelMSE.add(Dense(16, activation=LeakyReLU()))
modelMSE.add(Dense(units = 1, activation=LeakyReLU()))
modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

In [50]:
historyMSE = modelMSE.fit(train_X, train_y, epochs=10, batch_size = 256, shuffle=False, verbose = 2)

Epoch 1/10
4575/4575 - 7s - loss: 17.9063 - 7s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 7s - loss: 11.6839 - 7s/epoch - 1ms/step
Epoch 3/10
4575/4575 - 7s - loss: 10.3142 - 7s/epoch - 1ms/step
Epoch 4/10
4575/4575 - 7s - loss: 9.7306 - 7s/epoch - 1ms/step
Epoch 5/10
4575/4575 - 6s - loss: 9.3872 - 6s/epoch - 1ms/step
Epoch 6/10
4575/4575 - 7s - loss: 9.1287 - 7s/epoch - 1ms/step
Epoch 7/10
4575/4575 - 7s - loss: 8.9117 - 7s/epoch - 1ms/step
Epoch 8/10
4575/4575 - 6s - loss: 8.7328 - 6s/epoch - 1ms/step
Epoch 9/10
4575/4575 - 6s - loss: 8.5928 - 6s/epoch - 1ms/step
Epoch 10/10
4575/4575 - 6s - loss: 8.4902 - 6s/epoch - 1ms/step


In [52]:
modelMSE.summary()

Model: "sequential_139"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_213 (Conv1D)         (None, 8, 8)              32        
                                                                 
 max_pooling1d_142 (MaxPool  (None, 2, 8)              0         
 ing1D)                                                          
                                                                 
 flatten_138 (Flatten)       (None, 16)                0         
                                                                 
 dense_276 (Dense)           (None, 16)                272       
                                                                 
 dense_277 (Dense)           (None, 1)                 17        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
______________________

In [51]:
test_predict_MSE = modelMSE.predict(val_x)

9149/9149 [==============================] - 9s 944us/step


In [53]:
i =(len(x_command)-time_steps) - val_x.shape[0]%(len(x_command)-time_steps) -1
mean_3std = 0
for j in range(len(test_predict_MSE)):
    if i%len(x_command) == 0:
        i = time_steps
    if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
            mean_3std += 1
    i += 1
mean_3std/len(test_predict_MSE)

0.9910746308422228

In [68]:
filter_size = [8,16]
kernel_size_1 = [3,4]
kernel_size_2 = [3,4]
pooling_size = [3,4]
output_dens = [8, 16]
#batch_size = [256]

In [69]:
best_skor = float('-inf')
best_filter = None
best_kernel_1 = None
best_kernel_2 = None
best_pool = None
#best_batch = None
best_out = None
#best_step = None
loop = 0
for filt in filter_size:
    for kernel_1 in kernel_size_1:
        for kernel_2 in kernel_size_2:
            for pool in pooling_size:
                for out in output_dens:
                    keras.utils.set_random_seed(7)
                    optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)
                    loop += 1

                    modelMSE = Sequential()
                    modelMSE.add(Input(shape=input_shape))
                    modelMSE.add(Conv1D(filters=filt, kernel_size=kernel_1,padding='same', activation=LeakyReLU()))
                    modelMSE.add(MaxPooling1D(pool_size=pool))
                    modelMSE.add(Flatten())
                    modelMSE.add(Dense(out, activation=LeakyReLU()))
                    modelMSE.add(Dense(units = 1, activation=LeakyReLU()))
                    modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

                    modelMSE.fit(train_X, train_y, epochs = 10, batch_size = 256, verbose = 2, shuffle = False)
                    test_predict_MSE = modelMSE.predict(val_x)



                    #zato sto smo uzeli 20% od kraja, ne pocinje indeks od samog pocetka rada masine
                    #radimo evaluaciju na mean 3 sigma testu
                    i =(len(x_command)-time_steps) - val_x.shape[0]%(len(x_command)-time_steps) -1
                    mean_3std = 0
                    for j in range(len(test_predict_MSE)):
                        if i%len(x_command) == 0:
                            i = time_steps
                        if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
                                mean_3std += 1
                        i += 1



                    if mean_3std > best_skor:
                        best_skor = mean_3std
                        best_filter = filt
                        best_kernel_1 = kernel_1
                        best_kernel_2 = kernel_2
                        best_pool = pool
                        #best_batch = batch
                        best_out = out
                        #best_step = step


                        print('best_skor: ', best_skor)
                        print('best_filter: ', best_filter)
                        print('best_kernel_1: ', best_kernel_1)
                        print('best_kernel_2: ', best_kernel_2)
                        print('best_out: ', best_out )
                        #print('best_batch', best_batch)
                        #print('best_step', best_step)
                        print('best_pool', best_pool)

                    print('loop: ', loop)

print('best_skor: ', best_skor)
print('best_filter: ', best_filter)
print('best_kernel_1: ', best_kernel_1)
print('best_kernel_2: ', best_kernel_2)
print('best_out: ', best_out )
#print('best_batch', best_batch)
#print('best_step', best_step)
print('best_pool', best_pool)

Epoch 1/10
4575/4575 - 8s - loss: 20.9319 - 8s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 7s - loss: 13.2181 - 7s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 7s - loss: 11.5941 - 7s/epoch - 2ms/step
Epoch 4/10
4575/4575 - 7s - loss: 10.7129 - 7s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 7s - loss: 10.1676 - 7s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 7s - loss: 9.7395 - 7s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 7s - loss: 9.3280 - 7s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 7s - loss: 8.9717 - 7s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 7s - loss: 8.6898 - 7s/epoch - 2ms/step
Epoch 10/10
4575/4575 - 7s - loss: 8.4913 - 7s/epoch - 2ms/step
9149/9149 [==============================] - 9s 1ms/step
best_skor:  290329
best_filter:  8
best_kernel_1:  3
best_kernel_2:  3
best_out:  8
best_pool 3
loop:  1
Epoch 1/10
4575/4575 - 8s - loss: 17.9063 - 8s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 7s - loss: 11.6839 - 7s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 7s - loss: 10.3142 - 7s/epoch - 2ms/step
Epoch 4/10

Epoch 8/10
4575/4575 - 7s - loss: 3.7992 - 7s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 7s - loss: 3.6997 - 7s/epoch - 2ms/step
Epoch 10/10
4575/4575 - 7s - loss: 3.6161 - 7s/epoch - 2ms/step
9149/9149 [==============================] - 9s 996us/step
loop:  12
Epoch 1/10
4575/4575 - 9s - loss: 17.5177 - 9s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 7s - loss: 8.6506 - 7s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 7s - loss: 6.3341 - 7s/epoch - 2ms/step
Epoch 4/10
4575/4575 - 7s - loss: 5.4062 - 7s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 7s - loss: 4.8768 - 7s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 7s - loss: 4.4527 - 7s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 7s - loss: 4.1118 - 7s/epoch - 1ms/step
Epoch 8/10
4575/4575 - 7s - loss: 3.8574 - 7s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 7s - loss: 3.6731 - 7s/epoch - 2ms/step
Epoch 10/10
4575/4575 - 7s - loss: 3.5215 - 7s/epoch - 2ms/step
9149/9149 [==============================] - 9s 993us/step
loop:  13
Epoch 1/10
4575/4575 - 8s - loss: 15.145

Epoch 5/10
4575/4575 - 8s - loss: 4.5556 - 8s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 7s - loss: 4.0223 - 7s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 7s - loss: 3.7081 - 7s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 7s - loss: 3.5406 - 7s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 8s - loss: 3.4331 - 8s/epoch - 2ms/step
Epoch 10/10
4575/4575 - 7s - loss: 3.3520 - 7s/epoch - 2ms/step
9149/9149 [==============================] - 9s 990us/step
loop:  24
Epoch 1/10
4575/4575 - 9s - loss: 14.0022 - 9s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 8s - loss: 6.6754 - 8s/epoch - 2ms/step
Epoch 3/10
4575/4575 - 7s - loss: 5.1551 - 7s/epoch - 2ms/step
Epoch 4/10
4575/4575 - 7s - loss: 4.4014 - 7s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 7s - loss: 3.9469 - 7s/epoch - 2ms/step
Epoch 6/10
4575/4575 - 7s - loss: 3.6236 - 7s/epoch - 2ms/step
Epoch 7/10
4575/4575 - 8s - loss: 3.3612 - 8s/epoch - 2ms/step
Epoch 8/10
4575/4575 - 8s - loss: 3.1822 - 8s/epoch - 2ms/step
Epoch 9/10
4575/4575 - 7s - loss: 3.0557 - 7s/e

In [70]:
best_skor/len(test_predict_MSE)

0.9916928825902357

In [72]:
keras.utils.set_random_seed(7)
optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

modelMSE = Sequential()
modelMSE.add(Input(shape=(time_steps, 1)))
modelMSE.add(Conv1D(filters=8, kernel_size=3,padding='same', activation=LeakyReLU()))
modelMSE.add(MaxPooling1D(pool_size=3))
modelMSE.add(Flatten())
modelMSE.add(Dense(8, activation=LeakyReLU()))
modelMSE.add(Dense(units = 1, activation=LeakyReLU()))
modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

In [73]:
historyMSE = modelMSE.fit(train_X, train_y, epochs=10, batch_size = 256, shuffle=False, verbose = 2)

Epoch 1/10
4575/4575 - 9s - loss: 20.9319 - 9s/epoch - 2ms/step
Epoch 2/10
4575/4575 - 7s - loss: 13.2181 - 7s/epoch - 1ms/step
Epoch 3/10
4575/4575 - 7s - loss: 11.5941 - 7s/epoch - 1ms/step
Epoch 4/10
4575/4575 - 7s - loss: 10.7129 - 7s/epoch - 2ms/step
Epoch 5/10
4575/4575 - 7s - loss: 10.1676 - 7s/epoch - 1ms/step
Epoch 6/10
4575/4575 - 7s - loss: 9.7395 - 7s/epoch - 1ms/step
Epoch 7/10
4575/4575 - 7s - loss: 9.3280 - 7s/epoch - 1ms/step
Epoch 8/10
4575/4575 - 7s - loss: 8.9717 - 7s/epoch - 1ms/step
Epoch 9/10
4575/4575 - 7s - loss: 8.6898 - 7s/epoch - 1ms/step
Epoch 10/10
4575/4575 - 7s - loss: 8.4913 - 7s/epoch - 1ms/step


In [75]:
modelMSE.summary()

Model: "sequential_206"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_313 (Conv1D)         (None, 8, 8)              32        
                                                                 
 max_pooling1d_209 (MaxPool  (None, 2, 8)              0         
 ing1D)                                                          
                                                                 
 flatten_205 (Flatten)       (None, 16)                0         
                                                                 
 dense_410 (Dense)           (None, 8)                 136       
                                                                 
 dense_411 (Dense)           (None, 1)                 9         
                                                                 
Total params: 177 (708.00 Byte)
Trainable params: 177 (708.00 Byte)
Non-trainable params: 0 (0.00 Byte)
______________

In [74]:
test_predict_MSE = modelMSE.predict(val_x)

9149/9149 [==============================] - 9s 975us/step


In [76]:
i =(len(x_command)-time_steps) - val_x.shape[0]%(len(x_command)-time_steps) -1
mean_3std = 0
for j in range(len(test_predict_MSE)):
    if i%len(x_command) == 0:
        i = time_steps
    if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
            mean_3std += 1
    i += 1
mean_3std/len(test_predict_MSE)

0.9916928825902357